### Perform topic modeling using LSA, LDA. Finally you should obtain chart of coherence score by the number of topics.

In [15]:
import operator
from string import punctuation
from collections import Counter

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import Sparse2Corpus
from gensim import corpora

from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

from textblob import TextBlob
import warnings

warnings.filterwarnings("ignore")

#### Dataset from Internet

##### LSA

In [5]:
dataset = fetch_20newsgroups(
    shuffle=True, random_state=42, remove=("headers", "footers", "quotes")
)
documents = dataset.data
df = pd.DataFrame({"document": documents})
df = df.iloc[:5000]
df.head()

,document
0,I was wondering if anyone out there could enli...
1,A fair number of brave souls who upgraded thei...
2,"well folks, my mac plus finally gave up the gh..."
3,\nDo you have Weitek's address/phone number? ...
4,"From article <C5owCB.n3p@world.std.com>, by to..."


In [9]:
tokenized_doc = df["document"].str.replace("[^a-zA-Z#]", " ")
lemmatizer = WordNetLemmatizer()


def clean_doc(text):
    # remove small words
    text = " ".join([w.lower() for w in text.split() if len(w) > 3])

    # remove stopwords
    stopwords_set = set(stopwords.words("english"))
    stopwords_and_punctuation = list(stopwords_set) + list(punctuation)
    text = word_tokenize(text)
    text = " ".join([word for word in text if word not in stopwords_and_punctuation])

    # lemmatize
    text = word_tokenize(text)
    lemmatized_text = []
    for word in text:
        lemmatized_text.append(lemmatizer.lemmatize(word))
    text = " ".join(lemmatized_text)
    return text


tokenized_doc = tokenized_doc.apply(clean_doc)
tokenized_doc

0       wondering anyone could enlighten day 2-door sp...
1       fair number brave soul upgraded clock oscillat...
2       well folk plus finally gave ghost weekend star...
3       weitek 's address/phone number like informatio...
4       article c5owcb.n3p world.std.com tombaker worl...
                              ...                        
4995                               claim system objective
4996    phenylketonuria disease body process phenylala...
4997    reply zazen austin.ibm.com welbon many atheist...
4998                                                     
4999    waterski bike turn handlebar left lean right l...
Name: document, Length: 5000, dtype: object

In [10]:
tfidf = TfidfVectorizer(
    stop_words="english", max_features=1000, max_df=0.5, smooth_idf=True
)

matrix = tfidf.fit_transform(tokenized_doc)

X = matrix.toarray()

In [11]:
svd_model = TruncatedSVD(
    n_components=5, algorithm="randomized", n_iter=100, random_state=42
)
svd_model.fit(X)
print("The number of topics chosen are", len(svd_model.components_))

The number of topics chosen are 5


In [12]:
terms = tfidf.get_feature_names_out()
topics = []

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
    topics.append("Topic " + str(i) + ": ")
    for t in sorted_terms:
        topics.append(t[0])

final_topic_list = [topics[i : i + 8] for i in range(0, len(topics), 8)]

for x in final_topic_list:
    print(x)

['Topic 0: ', 'like', 'know', 'people', 'think', 'time', 'good', 'year']
['Topic 1: ', 'window', 'file', 'drive', 'thanks', 'card', 'program', 'problem']
['Topic 2: ', 'game', 'drive', 'team', 'player', 'year', 'play', 'season']
['Topic 3: ', 'drive', 'scsi', 'problem', 'people', 'hard', 'disk', 'christian']
['Topic 4: ', 'file', 'window', 'game', 'problem', 'think', 'team', 'program']


##### LDA

In [19]:
stopwords_set = set(stopwords.words("english"))
punctuation_set = set(punctuation)
lemmatizer = WordNetLemmatizer()


# Function to lemmatize and remove the stopwords
def clean_doc(doc):
    without_stopwords = " ".join(
        [i for i in doc.lower().split() if i not in stopwords_set]
    )
    without_puntuaction = "".join(
        ch for ch in without_stopwords if ch not in punctuation_set
    )
    normalized = " ".join(
        lemmatizer.lemmatize(word) for word in without_puntuaction.split()
    )
    return normalized


tokenized_doc = df["document"].str.replace("[^a-zA-Z#]", " ")
list_of_docs = tokenized_doc.tolist()
doc_clean = [clean_doc(doc).split() for doc in list_of_docs]

In [20]:
dictionary = corpora.Dictionary(doc_clean)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

ldamodel = LdaModel(
    corpus=doc_term_matrix, num_topics=5, id2word=dictionary, random_state=42, passes=30
)

pprint(ldamodel.print_topics())

[(0,
  '0.011*"armenian" + 0.006*"turkish" + 0.005*"jew" + 0.005*"israel" + '
  '0.004*"israeli" + 0.003*"turkey" + 0.003*"muslim" + 0.003*"russian" + '
  '0.003*"village" + 0.003*"arab"'),
 (1,
  '0.008*"would" + 0.008*"one" + 0.006*"people" + 0.005*"like" + 0.004*"think" '
  '+ 0.004*"know" + 0.004*"it" + 0.004*"get" + 0.004*"time" + 0.004*"well"'),
 (2,
  '0.027*"maxaxaxaxaxaxaxaxaxaxaxaxaxaxax" + 0.021*"x" + 0.007*"file" + '
  '0.006*"window" + 0.005*"use" + 0.004*"program" + 0.004*"image" + '
  '0.003*"system" + 0.003*"one" + 0.003*"widget"'),
 (3,
  '0.028*"1" + 0.014*"2" + 0.013*"0" + 0.009*"3" + 0.009*"game" + 0.008*"db" + '
  '0.006*"4" + 0.006*"team" + 0.006*"5" + 0.005*"play"'),
 (4,
  '0.006*"file" + 0.006*"system" + 0.006*"key" + 0.006*"space" + '
  '0.005*"information" + 0.004*"internet" + 0.004*"computer" + 0.004*"privacy" '
  '+ 0.004*"email" + 0.004*"anonymous"')]


##### Coherence score

In [21]:
# Compute Perplexity
perplexity_lda = ldamodel.log_perplexity(doc_term_matrix)
print("\nPerplexity: ", perplexity_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(
    model=ldamodel, texts=doc_clean, dictionary=dictionary, coherence="c_v"
)
coherence_lda = coherence_model_lda.get_coherence()
print("\nCoherence Score: ", coherence_lda)


Perplexity:  -8.90309896669351

Coherence Score:  0.6271698531660788


In [22]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
plot = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
plot

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.149782  0.020174       1        1  54.441556
2      0.159395 -0.066052       2        1  19.643000
4      0.058568  0.039966       3        1  10.346493
3     -0.200650 -0.183395       4        1   9.232995
0     -0.167094  0.189308       5        1   6.335956, topic_info=                                 Term         Freq        Total Category  \
6158  maxaxaxaxaxaxaxaxaxaxaxaxaxaxax  2896.000000  2896.000000  Default   
459                                 1  1909.000000  1909.000000  Default   
1427                                x  2239.000000  2239.000000  Default   
816                                 2  1217.000000  1217.000000  Default   
4312                                0   800.000000   800.000000  Default   
...                               ...          ...          ...      ...   
368                              said    70.344093   618.940738   Topic5   
1353                       university    59.177861   256.347367   Topic5   
2565                            woman    56.781473   187.164853   Topic5   
1935                       government    61.600666   567.740396   Topic5   
609                               the    61.028306   562.592121   Topic5   

      logprob  loglift  
6158  30.0000  30.0000  
459   29.0000  29.0000  
1427  28.0000  28.0000  
816   27.0000  27.0000  
4312  26.0000  26.0000  
...       ...      ...  
368   -6.1901   0.5843  
1353  -6.3629   1.2929  
2565  -6.4042   1.5661  
1935  -6.3228   0.5379  
609   -6.3321   0.5377  

[352 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
4312      2  0.173566     0
4312      4  0.820380     0
4312      5  0.006243     0
7604      4  0.991260    00
5316      4  0.958090  0000
...     ...       ...   ...
40        4  0.078565  year
40        5  0.015890  year
1371      1  0.954565   you
1371      2  0.034296   you
1371      3  0.011432   you

[690 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 4, 1])

#### Local dataset
##### LSA

In [35]:
df = pd.read_csv("small_wikidump.csv")
df = df.sample(5000, random_state=42)
df.head()

,id,url,title,text
70985,835014,https://simple.wikipedia.org/wiki/Humraaz,Humraaz,Humraaz () is a 2002 Indian Hindi-language mus...
175490,674066,https://simple.wikipedia.org/wiki/Yehoshua%20G...,Yehoshua Glazer,Yehoshua Glazer (29 December 1927 - 29 Decembe...
158572,664314,https://simple.wikipedia.org/wiki/2018%20Leice...,2018 Leicester helicopter crash,"On 27 October 2018, an AgustaWestland AW169 he..."
82591,658224,https://simple.wikipedia.org/wiki/Randolph%20C...,"Randolph County, Illinois",Randolph County is a county in the U.S. state ...
134171,508164,https://simple.wikipedia.org/wiki/The%20Real%2...,The Real Adventures of Jonny Quest,The Real Adventures of Jonny Quest is an Ameri...


In [36]:
tokenized_doc = df["text"].str.replace("[^a-zA-Z#]", " ")
lemmatizer = WordNetLemmatizer()


def clean_doc(text):
    # remove small words
    text = " ".join([w.lower() for w in text.split() if len(w) > 3])

    # remove stopwords
    stopwords_set = set(stopwords.words("english"))
    stopwords_and_punctuation = list(stopwords_set) + list(punctuation)
    text = word_tokenize(text)
    text = " ".join([word for word in text if word not in stopwords_and_punctuation])

    # lemmatize
    text = word_tokenize(text)
    lemmatized_text = []
    for word in text:
        lemmatized_text.append(lemmatizer.lemmatize(word))
    text = " ".join(lemmatized_text)
    return text


tokenized_doc = tokenized_doc.apply(clean_doc)
tokenized_doc

70985     humraaz 2002 indian hindi-language musical rom...
175490    yehoshua glazer december 1927 december 2018 is...
158572    october 2018 agustawestland aw169 helicopter c...
82591     randolph county county u.s. state illinois 202...
134171    real adventure jonny quest american animated a...
                                ...                        
196290    edelweiss well-known europe mountain flower de...
208656    viva zalata 1976 egyptian western comedy movie...
7584      husavik town northern iceland population 2,237...
16636     international astronomical union iau internati...
22733     girl group music group group female singer nor...
Name: text, Length: 5000, dtype: object

In [37]:
tfidf = TfidfVectorizer(
    stop_words="english", max_features=1000, max_df=0.5, smooth_idf=True
)

matrix = tfidf.fit_transform(tokenized_doc)

X = matrix.toarray()

In [38]:
svd_model = TruncatedSVD(
    n_components=5, algorithm="randomized", n_iter=100, random_state=42
)
svd_model.fit(X)
print("The number of topics chosen are", len(svd_model.components_))

The number of topics chosen are 5


In [39]:
terms = tfidf.get_feature_names_out()
topics = []

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
    topics.append("Topic " + str(i) + ": ")
    for t in sorted_terms:
        topics.append(t[0])

final_topic_list = [topics[i : i + 8] for i in range(0, len(topics), 8)]

for x in final_topic_list:
    print(x)

['Topic 0: ', 'commune', 'france', 'department', 'region', 'county', 'city', 'north']
['Topic 1: ', 'movie', 'county', 'city', 'state', 'american', 'actor', 'united']
['Topic 2: ', 'movie', 'actor', 'american', 'directed', 'television', 'comedy', 'drama']
['Topic 3: ', 'municipality', 'district', 'canton', 'switzerland', 'website', 'germany', 'province']
['Topic 4: ', 'movie', 'county', 'directed', 'drama', 'comedy', 'town', 'municipality']


##### LDA

In [41]:
stopwords_set = set(stopwords.words("english"))
punctuation_set = set(punctuation)
lemmatizer = WordNetLemmatizer()


# Function to lemmatize and remove the stopwords
def clean_doc(doc):
    without_stopwords = " ".join(
        [i for i in doc.lower().split() if i not in stopwords_set]
    )
    without_puntuaction = "".join(
        ch for ch in without_stopwords if ch not in punctuation_set
    )
    normalized = " ".join(
        lemmatizer.lemmatize(word) for word in without_puntuaction.split()
    )
    return normalized


tokenized_doc = df["text"].str.replace("[^a-zA-Z#]", " ")
list_of_docs = tokenized_doc.tolist()
doc_clean = [clean_doc(doc).split() for doc in list_of_docs]

In [42]:
dictionary = corpora.Dictionary(doc_clean)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

ldamodel = LdaModel(
    corpus=doc_term_matrix, num_topics=5, id2word=dictionary, random_state=42, passes=30
)

pprint(ldamodel.print_topics())

[(0,
  '0.008*"county" + 0.007*"state" + 0.006*"people" + 0.006*"also" + '
  '0.005*"united" + 0.005*"city" + 0.005*"used" + 0.005*"one" + '
  '0.004*"reference" + 0.004*"many"'),
 (1,
  '0.012*"american" + 0.011*"movie" + 0.005*"actor" + 0.005*"death" + '
  '0.005*"reference" + 0.004*"born" + 0.004*"birth" + 0.004*"television" + '
  '0.004*"new" + 0.004*"people"'),
 (2,
  '0.016*"commune" + 0.013*"municipality" + 0.011*"reference" + '
  '0.010*"district" + 0.009*"department" + 0.008*"region" + 0.008*"france" + '
  '0.007*"website" + 0.006*"language" + 0.004*"north"'),
 (3,
  '0.068*"km" + 0.068*"alignright" + 0.042*"linear" + 0.042*"socorro" + '
  '0.036*"bgcolorfefefe" + 0.017*"bgcolore9e9e9" + 0.015*"2001" + 0.015*"1999" '
  '+ 0.015*"2000" + 0.013*"bgcolord6d6d6"'),
 (4,
  '0.009*"player" + 0.008*"reference" + 0.007*"team" + 0.007*"league" + '
  '0.006*"football" + 0.005*"people" + 0.005*"birth" + 0.005*"national" + '
  '0.005*"world" + 0.005*"played"')]


##### Coherence score

In [43]:
# Compute Perplexity
perplexity_lda = ldamodel.log_perplexity(doc_term_matrix)
print("\nPerplexity: ", perplexity_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(
    model=ldamodel, texts=doc_clean, dictionary=dictionary, coherence="c_v"
)
coherence_lda = coherence_model_lda.get_coherence()
print("\nCoherence Score: ", coherence_lda)


Perplexity:  -8.719808438408297

Coherence Score:  0.6213836427516055


In [44]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
plot = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
plot

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.127264  0.124631       1        1  35.253005
0     -0.192025  0.039245       2        1  34.154344
4     -0.050262 -0.012595       3        1  12.994461
3      0.326729  0.103000       4        1  12.506244
2      0.042822 -0.254282       5        1   5.091946, topic_info=               Term         Freq        Total Category  logprob  loglift
860              km  5168.000000  5168.000000  Default  30.0000  30.0000
8279     alignright  5108.000000  5108.000000  Default  29.0000  29.0000
9412         linear  3174.000000  3174.000000  Default  28.0000  28.0000
9489        socorro  3161.000000  3161.000000  Default  27.0000  27.0000
8298  bgcolorfefefe  2748.000000  2748.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
115            city    82.080417  1424.505433   Topic5  -5.9278   0.1236
2421           west    58.519306   319.753053   Topic5  -6.2662   1.2793
2294           area    61.911974   520.601913   Topic5  -6.2098   0.8482
1583         german    57.173409   407.025370   Topic5  -6.2894   1.0147
1225       official    52.826869   413.712998   Topic5  -6.3685   0.9194

[358 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
2760       1  0.212606     12
2760       2  0.090670     12
2760       3  0.114119     12
2760       4  0.581539     12
2296       1  0.180826     13
...      ...       ...    ...
79         1  0.370427   year
79         2  0.441556   year
79         3  0.188447   year
9535       4  0.981852  yeung
17918      5  0.939558  yonne

[625 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 4, 3])